In [ ]:
from pprint import pprint as pprint
import numpy as np
import matplotlib.pyplot as plt

import control as pc

%pylab inline

import pandas as pd

from sysident import loadtools

import scipy as sp

In [ ]:
folder = "/home/lth/jupyter_nb/optimization/models/"
#fname = folder+"ss1_20180724-082630_poles2_ident_pade1_0:036_control_20180724-092309.npy"
#fname = folder+"ss2_20180724-082631_poles3_ident_pade2_0:036_control_20180724-092409.npy"
#fname = folder+"ss3_position_20180717-104106_poles3_ident_pade1_0:032_control_20180724-092445.npy"
fname = folder+"ss5_20180724-082628_poles3_ident_pade1_0:032_control_20180724-092522.npy"
#fname = folder+"ss6_20180724-082629_poles4_ident_pade2_0:032_control_20180724-092618.npy"

res, _ = loadtools.loadNPY(fname)
pprint(res.keys())

In [ ]:
#1.00807778e+00 8.44491407e+01 3.30837215e-04 4.67498491e-02

K = 1
D = 85
T = 3e-04

Td = 4e-02
Td_o = 10

sys0 = pc.tf([K],[T**2, 2*D*T, 1])
sys = pc.series(pc.tf(*pc.pade(Td, Td_o)), sys0)

t, y = pc.step_response(sys)
print "poles", pc.pole(sys0)

plt.plot(t, y)

#plt.plot(*pc.step_response(sys))
#plt.plot(*res['plot_model_step'])

plt.plot(*res['plot_input_step'])

plt.show()


df1 = pd.DataFrame(res['plot_input_step'][1], index=res['plot_input_step'][0])
df2 = pd.DataFrame(y, index=t)

df3 = pd.concat([df1, df2], axis=1).interpolate()

#df3.diff(axis=1).plot()
df3.diff(axis=1).abs().cumsum().plot()
#plt.plot(res['plot_input_step'][0])
print np.array(df3.diff(axis=1).abs().cumsum())[-1][1]

In [ ]:
def get_systems(K, T, D, Td, Td_o):
    sys0 = pc.tf([K],[T**2, 2*D*T, 1])
    sys = pc.series(pc.tf(*pc.pade(Td, Td_o)), sys0)
    return sys0, sys

In [ ]:
from scipy.optimize import minimize

def fun(arg, Td_o=1, show=False):
    
    K, D, T, Td = arg
    #print K, D, T, Td
    #T = np.abs(T)
    
    sys0, sys = get_systems(K, T, D, Td, Td_o)
    
    t, y = pc.step_response(sys)
    
    df1 = pd.DataFrame(res['plot_input_step'][1], index=res['plot_input_step'][0])
    df2 = pd.DataFrame(y, index=t)

    df3 = pd.concat([df1, df2], axis=1).interpolate()

    r = np.array(df3.diff(axis=1).abs().cumsum())[-1][1]
    
    if show:
        plt.plot(t, y)
        plt.plot(*res['plot_input_step'])
        plt.show()

        df3.diff(axis=1).plot()
        print r, '>>', arg
    return r


In [ ]:
result = minimize(fun, [1, 85, 0.00035, 0.04], method='Powell')
fun(result.x, show=True)

K, D, T, Td = result.x
Td_o = 1 #!!!!!!!!!!!!!!!!

sys0, sys = get_systems(K, T, D, Td, Td_o)
print pc.pole(sys)

In [ ]:
df1 = pd.DataFrame(res['plot_input_step'][1], index=res['plot_input_step'][0])
df2 = pd.DataFrame(res['plot_model_step'][1], index=res['plot_model_step'][0])

df3 = pd.concat([df1, df2], axis=1).interpolate()

plt.plot(*res['plot_model_step'])
plt.plot(*res['plot_input_step'])
plt.show()

df3.diff(axis=1).plot()
r = np.array(df3.diff(axis=1).abs().cumsum())[-1][1]
print r

In [ ]:
K, D, T, Td = result.x
Td_o = 1 #!!!!!!!!!!!!!!!!




sys0, sys = get_systems(K, T, D, Td, Td_o)
ss = pc.tf2ss(sys)

fname = 'ss_{}_poles{}_manual_ident_{}'.format(time.strftime("%Y%m%d-%H%M%S"),
                                    len(pc.pole(sys)), 'pade{}_{}'.format(Td_o, Td))


loadtools.saveDelayModel(fname, ss.A, ss.B, ss.C, ss.D, Td)